<a href="https://colab.research.google.com/github/syedwasif18/cifar10-image-classification/blob/main/Image_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
labels = pd.read_csv("D:/cifar-10/trainLabels.csv")
train_images_dir = 'D:/cifar-10/train/train'
test_images_dir = 'D:/cifar-10/test/test'

In [ ]:
label_map = {label: idx for idx, label in enumerate(labels['label'].unique())}
labels['label'] = labels['label'].map(label_map)

In [ ]:
import os

In [ ]:
def load_images(image_dir, image_size=(32, 32)):
    image_paths = [os.path.join(image_dir, img_file) for img_file in os.listdir(image_dir)]

    def _load_and_preprocess_image(image_path):
        img = tf.io.read_file(image_path)
        img = tf.image.decode_png(img, channels=3)
        img = tf.image.resize(img, image_size)
        img = img / 255.0
        return img

    images = [tf.numpy_function(_load_and_preprocess_image, [path], tf.float32) for path in image_paths]
    images = tf.stack(images)

    image_ids = [int(os.path.splitext(os.path.basename(path))[0]) for path in image_paths]
    image_ids = np.array(image_ids)

    return images, image_ids

In [ ]:
train_images, train_image_ids = load_images(train_images_dir)

In [ ]:
train_data = pd.DataFrame({'id': train_image_ids, 'image': list(train_images.numpy())})
train_data = pd.merge(train_data, labels, on='id')

In [ ]:
X_train = np.array(train_data['image'].tolist())
y_train = np.array(train_data['label'])

In [ ]:
y_train_encoded = to_categorical(y_train)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_encoded, test_size=0.2, random_state=42)

In [ ]:
model = Sequential([
    Flatten(input_shape=(32, 32, 3)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')  # 10 classes for CIFAR-10
])

C:\Users\wasif\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=64)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.2585 - loss: 2.0424 - val_accuracy: 0.3674 - val_loss: 1.7840
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.3710 - loss: 1.7542 - val_accuracy: 0.3907 - val_loss: 1.7298
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.4024 - loss: 1.6695 - val_accuracy: 0.4136 - val_loss: 1.6530
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.4219 - loss: 1.6160 - val_accuracy: 0.4204 - val_loss: 1.6357
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.4340 - loss: 1.5785 - val_accuracy: 0.4426 - val_loss: 1.5864
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.4476 - loss: 1.5455 - val_accuracy: 0.4358 - val_loss: 1.6106
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.4528 - loss: 1.5322 - val_accuracy: 0.4520 - val_loss: 1.5554
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.4644 - loss: 1.5061 - val_accuracy: 0.

In [ ]:
test_images, test_image_ids = load_images(test_images_dir)

In [ ]:
test_images_normalized = test_images / 255.0
y_pred = model.predict(test_images_normalized)
y_pred_labels = np.argmax(y_pred, axis=1)

315/315 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [ ]:
# Save the predictions in a CSV file
predictions = pd.DataFrame({'id': test_image_ids, 'label': y_pred_labels})
predictions.to_csv("D:/cifar-10/predictions.csv", index=False)

print("Predictions saved to predictions.csv.")

Predictions saved to predictions.csv.


In [ ]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4540 - loss: 1.5248
Validation Accuracy: 0.454800009727478


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report


y_val_pred = np.argmax(model.predict(X_val), axis=1)
y_val_true = np.argmax(y_val, axis=1)

accuracy = np.mean(y_val_pred == y_val_true)
precision = precision_score(y_val_true, y_val_pred, average='macro')
recall = recall_score(y_val_true, y_val_pred, average='macro')
f1 = f1_score(y_val_true, y_val_pred, average='macro')
conf_matrix = confusion_matrix(y_val_true, y_val_pred)
class_report = classification_report(y_val_true, y_val_pred)

# Print performance metrics
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Validation Accuracy: 0.4548
Precision: 0.4611
Recall: 0.4530
F1-Score: 0.4435
Confusion Matrix:
[[548  42 252  19  31  47  20  60  27   7]
 [ 20 612  33 154  13  40  77  14  23  29]
 [100  43 551  14  63  97  27  22  21  41]
 [ 28 202  48 542  10  23  93  13   9  28]
 [123  30 349  32 187  80  44  43  48  55]
 [ 25  71 181  16  40 524  26  26  32  30]
 [ 13 102  58  54  14  15 681  20  17  84]
 [162  59 159  39  74  96  61 216 107  26]
 [141  38 164  16  67 105  42 148 250  10]
 [ 17  95  66  28  59  47 178  15  15 437]]
Classification Report:
              precision    recall  f1-score   support

           0       0.47      0.52      0.49      1053
           1       0.47      0.60      0.53      1015
           2       0.30      0.56      0.39       979
           3       0.59      0.54      0.57       996
           4       0.34      0.19      0.24       991
           5       0.49      0.54      0.51       971
           6       0.55      0